In [1]:
pip install transformers

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.0 MB 2.4 MB/s eta 0:00:04
   ----- ---------------------------------- 1.3/10.0 MB 2.5 MB/s eta 0:00:04
   -------- ------------------------------- 2.1/10.0 MB 3.0 MB/s eta 0:00:03
   ------------ --------------------------- 3.1/10.0 MB 3.4 MB/s eta 0:00:03
   ---------------- ----------------------- 4.2/10.0 MB 3.8 MB/s eta 0:00:02
   -------------------- ------------------- 5.2/10.0 MB 4.0 MB/s eta 0:00:02
   ------------------------- -------------- 6.3/10.0 MB 4.1 MB/s eta 0:00:01
   ---------------------------- ----------- 7.1/10.0 MB 4.3 MB/s eta 0:00:01
   -------------------------------- ------- 8.1/10.0 MB 4.4 MB/s eta 0:00:01
   ------------------------------------- -- 9.4/10.0 MB 4.4 MB/s eta 0:00:01
   -----------------

In [5]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.7 GB 11.2 MB/s eta 0:04:02
     ---------------------------------------- 0.0/2.7 GB 11.2 MB/s eta 0:04:01
     ---------------------------------------- 0.0/2.7 GB 10.6 MB/s eta 0:04:16
     ---------------------------------------- 0.0/2.7 GB 10.6 MB/s eta 0:04:16
     ---------------------------------------- 0.0/2.7 GB 7.8 MB/s eta 0:05:45
     ---------------------------------------- 0.0/2.7 GB 7.8 MB/s eta 0:05:45
     ---------------------------------------- 0.0/2.7 GB 5.9 MB/s eta 0:07:34
     ---------------------------------------- 0.0/2.7 GB 5.9 MB/s eta 0:07:34
     ---------------------------------------- 0.0/2.7 GB 5.9 MB/s eta 0:07:34
     ---------------------------------------- 0.0/2.7 GB 5.9 MB/s eta 0:07:34
     ---------------------------------------- 0.0/2.7 GB 3.8 MB/s eta 0:11:41
     ---

In [3]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install frozendict

Note: you may need to restart the kernel to use updated packages.


## Dataset Loading

In [122]:
import requests, zipfile, io
import os

def download_files(dir_name):
  if not os.path.exists(dir_name): 
    # 28 training examples from the SQA training set + table csv data
    urls = ["https://www.dropbox.com/s/2p6ez9xro357i63/sqa_train_set_28_examples.zip?dl=1",
            "https://www.dropbox.com/s/abhum8ssuow87h6/table_csv.zip?dl=1"
    ]
    for url in urls:
      r = requests.get(url)
      z = zipfile.ZipFile(io.BytesIO(r.content))
      z.extractall()

dir_name = "sqa_data"
download_files(dir_name)

In [123]:
import pandas as pd

data = pd.read_excel("sqa_train_set_28_examples.xlsx")
data.head()

,id,annotator,position,question,table_file,answer_coordinates,answer_text
0,nt-639,0,0,where are the players from?,table_csv/203_149.csv,"['(0, 4)', '(1, 4)', '(2, 4)', '(3, 4)', '(4, ...","['Louisiana State University', 'Valley HS (Las..."
1,nt-639,0,1,which player went to louisiana state university?,table_csv/203_149.csv,"['(0, 1)']",['Ben McDonald']
2,nt-639,1,0,who are the players?,table_csv/203_149.csv,"['(0, 1)', '(1, 1)', '(2, 1)', '(3, 1)', '(4, ...","['Ben McDonald', 'Tyler Houston', 'Roger Salke..."
3,nt-639,1,1,which ones are in the top 26 picks?,table_csv/203_149.csv,"['(0, 1)', '(1, 1)', '(2, 1)', '(3, 1)', '(4, ...","['Ben McDonald', 'Tyler Houston', 'Roger Salke..."
4,nt-639,1,2,"and of those, who is from louisiana state univ...",table_csv/203_149.csv,"['(0, 1)']",['Ben McDonald']


### However, the answer_coordinates and answer_text columns are currently not recognized as real Python lists of Python tuples and strings respectively. Let's do that first using the .literal_eval()function of the ast module:

In [124]:
import ast

def _parse_answer_coordinates(answer_coordinate_str):
  """Parses the answer_coordinates of a question.
  Args:
    answer_coordinate_str: A string representation of a Python list of tuple
      strings.
      For example: "['(1, 4)','(1, 3)', ...]"
  """

  try:
    answer_coordinates = []
    # make a list of strings
    coords = ast.literal_eval(answer_coordinate_str)
    # parse each string as a tuple
    for row_index, column_index in sorted(
        ast.literal_eval(coord) for coord in coords):
      answer_coordinates.append((row_index, column_index))
  except SyntaxError:
    raise ValueError('Unable to evaluate %s' % answer_coordinate_str)
  
  return answer_coordinates


def _parse_answer_text(answer_text):
  """Populates the answer_texts field of `answer` by parsing `answer_text`.
  Args:
    answer_text: A string representation of a Python list of strings.
      For example: "[u'test', u'hello', ...]"
    answer: an Answer object.
  """
  try:
    answer = []
    for value in ast.literal_eval(answer_text):
      answer.append(value)
  except SyntaxError:
    raise ValueError('Unable to evaluate %s' % answer_text)

  return answer

data['answer_coordinates'] = data['answer_coordinates'].apply(lambda coords_str: _parse_answer_coordinates(coords_str))
data['answer_text'] = data['answer_text'].apply(lambda txt: _parse_answer_text(txt))

data.head(10)

,id,annotator,position,question,table_file,answer_coordinates,answer_text
0,nt-639,0,0,where are the players from?,table_csv/203_149.csv,"[(0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4...","[Louisiana State University, Valley HS (Las Ve..."
1,nt-639,0,1,which player went to louisiana state university?,table_csv/203_149.csv,"[(0, 1)]",[Ben McDonald]
2,nt-639,1,0,who are the players?,table_csv/203_149.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Ben McDonald, Tyler Houston, Roger Salkeld, J..."
3,nt-639,1,1,which ones are in the top 26 picks?,table_csv/203_149.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Ben McDonald, Tyler Houston, Roger Salkeld, J..."
4,nt-639,1,2,"and of those, who is from louisiana state univ...",table_csv/203_149.csv,"[(0, 1)]",[Ben McDonald]
5,nt-639,2,0,who are the players in the top 26?,table_csv/203_149.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Ben McDonald, Tyler Houston, Roger Salkeld, J..."
6,nt-639,2,1,"of those, which one was from louisiana state u...",table_csv/203_149.csv,"[(0, 1)]",[Ben McDonald]
7,nt-11649,0,0,what are all the names of the teams?,table_csv/204_135.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Cordoba CF, CD Malaga, Granada CF, UD Las Pal..."
8,nt-11649,0,1,"of these, which teams had any losses?",table_csv/204_135.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Cordoba CF, CD Malaga, Granada CF, UD Las Pal..."
9,nt-11649,0,2,"of these teams, which had more than 21 losses?",table_csv/204_135.csv,"[(15, 1)]",[CD Villarrobledo]


### creating new dataframe

In [125]:
def get_sequence_id(example_id, annotator):
  if "-" in str(annotator):
    raise ValueError('"-" not allowed in annotator.')
  return f"{example_id}-{annotator}"

data['sequence_id'] = data.apply(lambda x: get_sequence_id(x.id, x.annotator), axis=1)
data.head()

,id,annotator,position,question,table_file,answer_coordinates,answer_text,sequence_id
0,nt-639,0,0,where are the players from?,table_csv/203_149.csv,"[(0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4...","[Louisiana State University, Valley HS (Las Ve...",nt-639-0
1,nt-639,0,1,which player went to louisiana state university?,table_csv/203_149.csv,"[(0, 1)]",[Ben McDonald],nt-639-0
2,nt-639,1,0,who are the players?,table_csv/203_149.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Ben McDonald, Tyler Houston, Roger Salkeld, J...",nt-639-1
3,nt-639,1,1,which ones are in the top 26 picks?,table_csv/203_149.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Ben McDonald, Tyler Houston, Roger Salkeld, J...",nt-639-1
4,nt-639,1,2,"and of those, who is from louisiana state univ...",table_csv/203_149.csv,"[(0, 1)]",[Ben McDonald],nt-639-1


In [126]:
# let's group table-question pairs by sequence id, and remove some columns we don't need 
grouped = data.groupby(by='sequence_id').agg(lambda x: x.tolist())
grouped = grouped.drop(columns=['id', 'annotator', 'position'])
grouped['table_file'] = grouped['table_file'].apply(lambda x: x[0])
grouped.head(10)

,question,table_file,answer_coordinates,answer_text
sequence_id,,,,
ns-1292-0,"[who are all the athletes?, where are they fro...",table_csv/204_521.csv,"[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, ...","[[Tommy Green, Janis Dalins, Ugo Frigerio, Kar..."
nt-10730-0,[what was the production numbers of each revol...,table_csv/203_253.csv,"[[(0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, ...","[[1,900 (estimated), 14,500 (estimated), 6,000..."
nt-10730-1,[what three revolver models had the least amou...,table_csv/203_253.csv,"[[(0, 0), (6, 0), (7, 0)], [(0, 0)]]","[[Remington-Beals Army Model Revolver, New Mod..."
nt-10730-2,"[what are all of the remington models?, how ma...",table_csv/203_253.csv,"[[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, ...","[[Remington-Beals Army Model Revolver, Remingt..."
nt-11649-0,"[what are all the names of the teams?, of thes...",table_csv/204_135.csv,"[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, ...","[[Cordoba CF, CD Malaga, Granada CF, UD Las Pa..."
nt-11649-1,"[what are the losses?, what team had more than...",table_csv/204_135.csv,"[[(0, 6), (1, 6), (2, 6), (3, 6), (4, 6), (5, ...","[[6, 6, 9, 10, 10, 12, 12, 11, 13, 14, 15, 14,..."
nt-11649-2,"[what were all the teams?, what were the loss ...",table_csv/204_135.csv,"[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, ...","[[Cordoba CF, CD Malaga, Granada CF, UD Las Pa..."
nt-639-0,"[where are the players from?, which player wen...",table_csv/203_149.csv,"[[(0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, ...","[[Louisiana State University, Valley HS (Las V..."
nt-639-1,"[who are the players?, which ones are in the t...",table_csv/203_149.csv,"[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, ...","[[Ben McDonald, Tyler Houston, Roger Salkeld, ..."


### Load the tables data

In [127]:
# path to the directory containing all csv files
table_csv_path = "table_csv"

item = grouped.iloc[0]
table = pd.read_csv(table_csv_path + item.table_file[9:]).astype(str) 

display(table)
print("")
print(item.question)
     

,Rank,Name,Nationality,Time (hand),Notes
0,nan,Tommy Green,Great Britain,4:50:10,OR
1,nan,Janis Dalins,Latvia,4:57:20,nan
2,nan,Ugo Frigerio,Italy,4:59:06,nan
3,4.0,Karl Hahnel,Germany,5:06:06,nan
4,5.0,Ettore Rivolta,Italy,5:07:39,nan
5,6.0,Paul Sievert,Germany,5:16:41,nan
6,7.0,Henri Quintric,France,5:27:25,nan
7,8.0,Ernie Crosbie,United States,5:28:02,nan
8,9.0,Bill Chisholm,United States,5:51:00,nan
9,10.0,Alfred Maasik,Estonia,6:19:00,nan



['who are all the athletes?', 'where are they from?', 'along with paul sievert, which athlete is from germany?']


## Load tokenizer

In [128]:
import torch
from transformers import TapasTokenizer

# initialize the tokenizer
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base")

## encoding

In [129]:
encoding = tokenizer(table=table, queries=item.question, answer_coordinates=item.answer_coordinates, answer_text=item.answer_text,
                     truncation=True, padding="max_length", return_tensors="pt")
encoding.keys()

C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\tapas\tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\tapas\tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


dict_keys(['input_ids', 'labels', 'numeric_values', 'numeric_values_scale', 'token_type_ids', 'attention_mask'])

#### The token_type_ids created here will be of shape (batch_size, sequence_length, 7), as TAPAS uses 7 different token types to encode tabular structure. Let's verify this:

In [130]:
assert encoding["token_type_ids"].shape == (3, 512, 7)

#### The prev_label token type ids of the first example in a batch must always be zero (since there's no previous table-question pair). Let's verify this:

In [131]:
assert encoding["token_type_ids"][0][:,3].sum() == 0

#### So let's now verify whether the prev_label ids of the second table-question pair are set correctly:

In [132]:

for id, prev_label in zip (encoding["input_ids"][1], encoding["token_type_ids"][1][:,3]):
  if id != 0: # we skip padding tokens
    print(tokenizer.decode([id]), prev_label.item())
     

[CLS] 0
where 0
are 0
they 0
from 0
? 0
[SEP] 0
rank 0
name 0
nationality 0
time 0
( 0
hand 0
) 0
notes 0
[EMPTY] 0
tommy 1
green 1
great 0
britain 0
4 0
: 0
50 0
: 0
10 0
or 0
[EMPTY] 0
jan 1
##is 1
dali 1
##ns 1
latvia 0
4 0
: 0
57 0
: 0
20 0
[EMPTY] 0
[EMPTY] 0
u 1
##go 1
fr 1
##iger 1
##io 1
italy 0
4 0
: 0
59 0
: 0
06 0
[EMPTY] 0
4 0
. 0
0 0
karl 1
hahn 1
##el 1
germany 0
5 0
: 0
06 0
: 0
06 0
[EMPTY] 0
5 0
. 0
0 0
et 1
##tore 1
ri 1
##vo 1
##lta 1
italy 0
5 0
: 0
07 0
: 0
39 0
[EMPTY] 0
6 0
. 0
0 0
paul 1
si 1
##ever 1
##t 1
germany 0
5 0
: 0
16 0
: 0
41 0
[EMPTY] 0
7 0
. 0
0 0
henri 1
qui 1
##nt 1
##ric 1
france 0
5 0
: 0
27 0
: 0
25 0
[EMPTY] 0
8 0
. 0
0 0
ernie 1
cr 1
##os 1
##bie 1
united 0
states 0
5 0
: 0
28 0
: 0
02 0
[EMPTY] 0
9 0
. 0
0 0
bill 1
chi 1
##sho 1
##lm 1
united 0
states 0
5 0
: 0
51 0
: 0
00 0
[EMPTY] 0
10 0
. 0
0 0
alfred 1
ma 1
##asi 1
##k 1
estonia 0
6 0
: 0
19 0
: 0
00 0
[EMPTY] 0
[EMPTY] 0
henry 1
ci 1
##eman 1
canada 0
[EMPTY] 0
d 0
##n 0
##f 0
[EMPTY] 0

## Table Loader Class

In [133]:
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        table = pd.read_csv(table_csv_path + item.table_file[9:]).astype(str) # TapasTokenizer expects the table data to be text only
        if item.position != 0:
          # use the previous table-question pair to correctly set the prev_labels token type ids
          previous_item = self.df.iloc[idx-1]
          encoding = self.tokenizer(table=table, 
                                    queries=[previous_item.question, item.question], 
                                    answer_coordinates=[previous_item.answer_coordinates, item.answer_coordinates], 
                                    answer_text=[previous_item.answer_text, item.answer_text],
                                    padding="max_length",
                                    truncation=True,
                                    return_tensors="pt"
          )
          # use encodings of second table-question pair in the batch
          encoding = {key: val[-1] for key, val in encoding.items()}
        else:
          # this means it's the first table-question pair in a sequence
          encoding = self.tokenizer(table=table, 
                                    queries=item.question, 
                                    answer_coordinates=item.answer_coordinates, 
                                    answer_text=item.answer_text,
                                    padding="max_length",
                                    truncation=True,
                                    return_tensors="pt"
          )
          # remove the batch dimension which the tokenizer adds 
          encoding = {key: val.squeeze(0) for key, val in encoding.items()}
        return encoding

    def __len__(self):
        return len(self.df)

train_dataset = TableDataset(df=data, tokenizer=tokenizer)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2)

In [136]:
batch = next(iter(train_dataloader))

C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\tapas\tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\tapas\tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\tapas\tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In

In [137]:
#first example should not have any prev_labels set
assert batch["token_type_ids"][0][:,3].sum() == 0

## Load model

In [138]:
from transformers import TapasForQuestionAnswering

model = TapasForQuestionAnswering.from_pretrained("google/tapas-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['column_output_bias', 'column_output_weights', 'output_bias', 'output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TapasForQuestionAnswering(
  (tapas): TapasModel(
    (embeddings): TapasEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (token_type_embeddings_0): Embedding(3, 768)
      (token_type_embeddings_1): Embedding(256, 768)
      (token_type_embeddings_2): Embedding(256, 768)
      (token_type_embeddings_3): Embedding(2, 768)
      (token_type_embeddings_4): Embedding(256, 768)
      (token_type_embeddings_5): Embedding(256, 768)
      (token_type_embeddings_6): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.07, inplace=False)
    )
    (encoder): TapasEncoder(
      (layer): ModuleList(
        (0-11): 12 x TapasLayer(
          (attention): TapasAttention(
            (self): TapasSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bia

## Traning Model

In [140]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):  # loop over the dataset multiple times
   print("Epoch:", epoch)
   for idx, batch in enumerate(train_dataloader):
        # get the inputs;
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch["labels"].to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,
                       labels=labels)
        loss = outputs.loss
        print("Loss:", loss.item())
        loss.backward()
        optimizer.step()

Epoch: 0
Loss: 2.1405766010284424
Loss: 1.9862993955612183
Loss: 1.928906798362732
Loss: 2.160287380218506
Loss: 2.4079952239990234
Loss: 2.689960479736328
Loss: 2.6415350437164307
Loss: 2.6616296768188477
Loss: 2.345637798309326
Loss: 2.117447853088379
Loss: 2.0566954612731934
Loss: 2.0776736736297607
Loss: 2.0250511169433594
Loss: 1.652886986732483
Epoch: 1
Loss: 1.9999897480010986
Loss: 1.6096112728118896
Loss: 1.5266094207763672
Loss: 1.7902915477752686
Loss: 2.0703561305999756
Loss: 2.2517998218536377
Loss: 2.2556214332580566
Loss: 2.2593154907226562
Loss: 2.025332450866699
Loss: 1.762717843055725
Loss: 1.7825895547866821
Loss: 1.8152310848236084
Loss: 1.7366342544555664
Loss: 1.5097631216049194
Epoch: 2
Loss: 1.8359665870666504
Loss: 1.3793697357177734
Loss: 1.278315544128418
Loss: 1.5381947755813599
Loss: 1.8301724195480347
Loss: 1.8544429540634155
Loss: 1.8729064464569092
Loss: 1.9565740823745728
Loss: 1.722538948059082
Loss: 1.587408423423767
Loss: 1.6065402030944824
Loss: 1.6

## Login on HuggingFace

In [146]:
from huggingface_hub import notebook_login

notebook_login()

## Push on huggingface

In [147]:
# Replace 'your-username' with your Hugging Face username and set the repository name
repo_name = "Tapas-For-QA_on_SQA"

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

SafetensorError: Error while serializing: IoError(Os { code: 112, kind: StorageFull, message: "There is not enough space on the disk." })

## Inference

In [141]:
import collections
import numpy as np

def compute_prediction_sequence(model, data, device):
  """Computes predictions using model's answers to the previous questions."""
  
  # prepare data
  input_ids = data["input_ids"].to(device)
  attention_mask = data["attention_mask"].to(device)
  token_type_ids = data["token_type_ids"].to(device)

  all_logits = []
  prev_answers = None

  num_batch = data["input_ids"].shape[0]
  
  for idx in range(num_batch):
    
    if prev_answers is not None:
        coords_to_answer = prev_answers[idx]
        # Next, set the label ids predicted by the model
        prev_label_ids_example = token_type_ids_example[:,3] # shape (seq_len,)
        model_label_ids = np.zeros_like(prev_label_ids_example.cpu().numpy()) # shape (seq_len,)

        # for each token in the sequence:
        token_type_ids_example = token_type_ids[idx] # shape (seq_len, 7)
        for i in range(model_label_ids.shape[0]):
          segment_id = token_type_ids_example[:,0].tolist()[i]
          col_id = token_type_ids_example[:,1].tolist()[i] - 1
          row_id = token_type_ids_example[:,2].tolist()[i] - 1
          if row_id >= 0 and col_id >= 0 and segment_id == 1:
            model_label_ids[i] = int(coords_to_answer[(col_id, row_id)])

        # set the prev label ids of the example (shape (1, seq_len) )
        token_type_ids_example[:,3] = torch.from_numpy(model_label_ids).type(torch.long).to(device)   

    prev_answers = {}
    # get the example
    input_ids_example = input_ids[idx] # shape (seq_len,)
    attention_mask_example = attention_mask[idx] # shape (seq_len,)
    token_type_ids_example = token_type_ids[idx] # shape (seq_len, 7)
    # forward pass to obtain the logits
    outputs = model(input_ids=input_ids_example.unsqueeze(0), 
                    attention_mask=attention_mask_example.unsqueeze(0), 
                    token_type_ids=token_type_ids_example.unsqueeze(0))
    logits = outputs.logits
    all_logits.append(logits)

    # convert logits to probabilities (which are of shape (1, seq_len))
    dist_per_token = torch.distributions.Bernoulli(logits=logits)
    probabilities = dist_per_token.probs * attention_mask_example.type(torch.float32).to(dist_per_token.probs.device) 

    # Compute average probability per cell, aggregating over tokens.
    # Dictionary maps coordinates to a list of one or more probabilities
    coords_to_probs = collections.defaultdict(list)
    prev_answers = {}
    for i, p in enumerate(probabilities.squeeze().tolist()):
      segment_id = token_type_ids_example[:,0].tolist()[i]
      col = token_type_ids_example[:,1].tolist()[i] - 1
      row = token_type_ids_example[:,2].tolist()[i] - 1
      if col >= 0 and row >= 0 and segment_id == 1:
        coords_to_probs[(col, row)].append(p)

    # Next, map cell coordinates to 1 or 0 (depending on whether the mean prob of all cell tokens is > 0.5)
    coords_to_answer = {}
    for key in coords_to_probs:
      coords_to_answer[key] = np.array(coords_to_probs[key]).mean() > 0.5
    prev_answers[idx+1] = coords_to_answer
    
  logits_batch = torch.cat(tuple(all_logits), 0)
  
  return logits_batch

In [142]:
data = {'Actors': ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], 
        'Age': ["56", "45", "59"],
        'Number of movies': ["87", "53", "69"],
        'Date of birth': ["7 february 1967", "10 june 1996", "28 november 1967"]}
queries = ["How many movies has George Clooney played in?", "How old is he?", "What's his date of birth?"]

table = pd.DataFrame.from_dict(data)

inputs = tokenizer(table=table, queries=queries, padding='max_length', return_tensors="pt")
logits = compute_prediction_sequence(model, inputs, device)

In [143]:
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())


In [144]:
# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(table.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(table.iat[coordinate])
    answers.append(", ".join(cell_values))

display(table)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)
     

,Actors,Age,Number of movies,Date of birth
0,Brad Pitt,56,87,7 february 1967
1,Leonardo Di Caprio,45,53,10 june 1996
2,George Clooney,59,69,28 november 1967



How many movies has George Clooney played in?
Predicted answer: 
How old is he?
Predicted answer: 56, 45, 59
What's his date of birth?
Predicted answer: 
